In [2]:
from sonosco.training import Experiment
from sonosco.common.path_utils import parse_yaml
from sonosco.datasets import create_data_loaders
import torch.nn as nn


In [7]:
class View(nn.Module):
    def __init__(self, shape):
        super(View, self).__init__()
        self.shape = shape
        
    def forward(self, input):
        return input.view(self.shape)

class Reorder(nn.Module):
    def __init__(self, shape):
        super(Reorder, self).__init__()
        self.shape = shape
        
    def forward(self, input):
        return input.permute(self.shape)

In [17]:
class TDSBlock(nn.Module):
    
    def __init__(self, c: int, kw: int, h: int, dropout: float)-> None:
        super().__init__()
        l:int = c * h
        self.conv = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=(21, 1), stride=(2, 1), padding=(20, 5)), #calc padding
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.LayerNorm(3),
            View((-1,l,1,0)),
            Reorder((1,0,2,3)),
            nn.Linear(l,l),
            nn.ReLU(),
            nn.Dropout2d(dropout),
            nn.Linear(l,l),
            Reorder((1,0,2,3)),
            View((-1,h,c,0)),
            nn.Dropout2d(dropout),
            nn.LayerNorm(3)
        )
        
    def forward(self, *input):
        return self.conv(input)

In [13]:
b = TDSBlock(1,2,3,0.5)


In [14]:
Experiment.create("CV_STS")
config = parse_yaml("./config.yaml")["train"]

In [15]:
train_loader, val_loader = create_data_loaders(**config)


In [18]:
net = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=(21, 1), stride=(2, 1), padding=(20, 5)), #calc padding
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.LayerNorm(3),
            TDSBlock(10,21,82,0.2),
            TDSBlock(10,21,82,0.2),
            nn.Conv2d(10, 14, kernel_size=(21, 1), stride=(2, 1), padding=(20, 5)), #calc padding
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.LayerNorm(3),
            TDSBlock(10,21,82,0.2),
            TDSBlock(10,21,82,0.2),
            TDSBlock(10,21,82,0.2),
            nn.Conv2d(14, 18, kernel_size=(21, 1), stride=(2, 1), padding=(20, 5)), #calc padding
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.LayerNorm(3),
            TDSBlock(10,21,82,0.2),
            TDSBlock(10,21,82,0.2),
            TDSBlock(10,21,82,0.2),
            TDSBlock(10,21,82,0.2),
            TDSBlock(10,21,82,0.2),
            TDSBlock(10,21,82,0.2),
            View((-1,1440,1,0)),
            Reorder((1,0,3,2)),
            nn.Linear(1440,1024),
        )